In [ ]:
!pip install -r requirements.txt

In [6]:
%cd ./tfx-pipeline
GOOGLE_CLOUD_PROJECT="or2--epm-gcp-by-meetup2-t1iylu"
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
# %env GOOGLE_APPLICATION_CREDENTIALS=/Users/oleg/Projects/EPAM/creds.json
%env UNSUPPORTED_DO_NOT_PACKAGE_USER_MODULES=0


from pipeline import configs, components
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx import v1 as tfx
from tfx.dsl.components.common.importer import Importer
context = InteractiveContext(
    pipeline_root=configs.PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(
              './metadata.db'
    )
)

[Errno 2] No such file or directory: './tfx-pipeline'
/home/jupyter/ml-specialization-demo2/tfx-pipeline
env: GOOGLE_CLOUD_PROJECT=or2--epm-gcp-by-meetup2-t1iylu
env: UNSUPPORTED_DO_NOT_PACKAGE_USER_MODULES=0


In [2]:
examples = Importer(
    source_uri='gs://epm-spec-black-friday/tfx_pipeline_output/tfx-pipeline-ml-demo2-black-friday/877218744686/tfx-pipeline-ml-demo2-black-friday-20211012171731/CsvExampleGen_-2668286022193774592/examples',
    artifact_type=tfx.types.standard_artifacts.Examples,
    reimport=True,
    properties={'split_names': '["train", "eval"]'}
)

context.run(examples)

examples=examples.outputs['result']
examples

Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 258
type_id: 14
uri: "gs://epm-spec-black-friday/tfx_pipeline_output/tfx-pipeline-ml-demo2-black-friday/877218744686/tfx-pipeline-ml-demo2-black-friday-20211012171731/CsvExampleGen_-2668286022193774592/examples"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.3.0"
  }
}
, artifact_type: id: 14
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
    additional_properties: {}
    additional_custom_properties: {}
)

In [ ]:
trainer = components.trainer_vertex(
    examples=examples,
#    schema_gen=schema_gen,
#    transform=transform
)
context.run(trainer)

In [ ]:
tuner = components.tuner(
    examples=examples,
#    schema_gen=schema_gen,
#    transform=transform
)
context.run(tuner)

In [7]:
deploy = components.finish_pubsub_event(topic='test_topic', project='peoject_id')
context.run(deploy)

Publishing to PubSub topic test_topic...


ExecutionResult(
    component_id: finish_pubsub_event
    execution_id: 176
    outputs:
)

ERROR:google.cloud.pubsub_v1.publisher._batch.thread:Failed to publish 1 messages.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/google/api_core/grpc_helpers.py", line 67, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/grpc/_channel.py", line 946, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/opt/conda/lib/python3.7/site-packages/grpc/_channel.py", line 849, in _end_unary_response_blocking
    raise _InactiveRpcError(state)
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "You have passed an invalid argument to the service (argument=peoject_id)."
	debug_error_string = "{"created":"@1634154839.566570272","description":"Error received from peer ipv4:172.217.212.95:443","file":"src/core/lib/surface/call.cc","file_line":1069,"grpc_message":"You have passed an invalid arg

In [ ]:
model = Importer(
    source_uri='gs://epm-spec-black-friday/tfx_pipeline_output/tfx-pipeline-ml-demo2-black-friday/TrainerVertex/model/165',
    artifact_type=tfx.types.standard_artifacts.Model,
    reimport=True)

context.run(model)

model=model.outputs['result']

pusher = components.pusher(
    model=model,
#    schema_gen=schema_gen,
#    transform=transform
)
context.run(pusher)

In [ ]:
!pip list | grep tfx